In [1]:
from pyomo.environ import *

In [2]:
A = ["LS","SB"]
profit = {"LS":24,"SB":40}

compA = {"LS":18,"SB":12}
compB = {"LS":6,"SB":10}

In [7]:
model = ConcreteModel()

model.x = Var(A, bounds = (0,None), within = IntegerSet)

model.obj_value = Objective(
    expr = sum(model.x[i] * profit[i] for i in A),
    sense = maximize
)

model.comp_a = Constraint(
    expr = sum(model.x[i] * compA[i] for i in A) <= 4000
)

model.comp_b = Constraint(
    expr = sum(model.x[i] * compB[i] for i in A) <= 3500
)

opt = SolverFactory("glpk")
result_obj = opt.solve(model)

model.pprint()

KeyError: 'T'

In [8]:
A = ["R","TV"]
expo = {"R":350, "TV":800}
budget = {"R":400,"TV":2000}
man = {"R":0.7,"TV":-0.3}

In [9]:
model = ConcreteModel()

model.x = Var(A, within = NonNegativeReals)

model.obj_value = Objective(
    expr = sum(model.x[i] * expo[i] for i in A),
    sense = maximize
)

model.b = Constraint(
    expr = sum(model.x[i] * budget[i] for i in A) <= 25000
)

model.m = Constraint(
    expr = sum(model.x[i] * man[i] for i in A) <= 0
)

opt = SolverFactory("glpk")
result_obj = opt.solve(model)

model.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['R', 'TV']

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          R :     0 : 4.93421052631579 :  None : False : False : NonNegativeReals
         TV :     0 : 11.5131578947368 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj_value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 350*x[R] + 800*x[TV]

2 Constraint Declarations
    b : Size=1, Index=None, Active=True
        Key  : Lower : Body                  : Upper   : Active
        None :  -Inf : 400*x[R] + 2000*x[TV] : 25000.0 :   True
    m : Size=1, Index=None, Active=True
        Key  : Lower : Body                 : Upper : Active
        None :  -Inf : 0.7*x[R] - 0.3*x[TV] :   0.0 :   True

5 Declarations: x_index x obj_value b m


In [11]:
A = ["hammer","wrench","screwdriver","towel"]
b = {"hammer":8,"wrench":3,"screwdriver":6,"towel":11}
w = {"hammer":5,"wrench":7,"screwdriver":4,"towel":3}

model = ConcreteModel() 
model.x = Var(A,within=Binary)
model.obj_val = Objective(
    expr = sum(model.x[i] * b[i] for i in A),
    sense=maximize
)
model.weight = Constraint(
    expr = sum(model.x[i] * b[i] for i in A) <= 14
)

opt = SolverFactory("glpk")
result_obj = opt.solve(model)

model.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['hammer', 'screwdriver', 'towel', 'wrench']

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   0.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj_val : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                       : Upper : Active
        None :  -Inf : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11

In [12]:
Demand = {
   'Lon':   125,        # London
   'Ber':   175,        # Berlin
   'Maa':   225,        # Maastricht
   'Ams':   250,        # Amsterdam
   'Utr':   225,        # Utrecht
   'Hag':   200         # The Hague
}

Supply = {
   'Arn':   600,        # Arnhem
   'Gou':   650         # Gouda
}

T = {
    ('Lon','Arn'): 1000,
    ('Lon','Gou'): 2.5,
    ('Ber','Arn'): 2.5,
    ('Ber','Gou'): 1000,
    ('Maa','Arn'): 1.6,
    ('Maa','Gou'): 2.0,
    ('Ams','Arn'): 1.4,
    ('Ams','Gou'): 1.0,
    ('Utr','Arn'): 0.8,
    ('Utr','Gou'): 1.0,
    ('Hag','Arn'): 1.4,
    ('Hag','Gou'): 0.8
}

In [17]:
d = list(Demand.keys())

In [18]:
s = list(Supply.keys())

In [24]:
model = ConcreteModel()

model.x = Var(d,s,within=NonNegativeReals)
model.obj_val = Objective(
    expr = sum(model.x[i,j]*T[i,j] for i in d for j in s),
    sense = minimize
)
model.capacity = ConstraintList()

for item in s:
    model.capacity.add(sum(model.x[i,item] for i in d)<=Supply[item])

    
model.supply=  ConstraintList()

for item in d:
    model.supply.add(sum(model.x[item,j] for j in s)>=Demand[item])

In [25]:
opt = SolverFactory("glpk")
result_obj = opt.solve(model)

model.pprint()

5 Set Declarations
    capacity_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        [1, 2]
    supply_index : Dim=0, Dimen=1, Size=6, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4, 5, 6]
    x_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=6, Domain=None, Ordered=False, Bounds=None
        ['Ams', 'Ber', 'Hag', 'Lon', 'Maa', 'Utr']
    x_index_1 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Arn', 'Gou']

1 Var Declarations
    x : Size=12, Index=x_index
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
        ('Ams', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Ams', 'Gou') :     0 : 250.0 :  None : False : False : NonNegativeReals
        ('Ber', 'Arn') :     0 : 175.0 :  None : False : False : NonNegativeReals
        ('Ber', 'Gou') :     0 :   0.0 :  None : False : False : NonNegat